In [2]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import dateutil.parser as parser
from datetime import datetime, date, timedelta
import torch
import skorch
import scipy
import torch.nn as nn
import torch
import torch.nn.functional as F
import sys
from skorch.helper import DataFrameTransformer
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score
from sklearn import metrics
from sklearn.preprocessing import FunctionTransformer
from skorch.callbacks import EarlyStopping
from sklearn.pipeline import Pipeline

# Please ignore the below cells, they were for additional preprocessing and do not need to be run again

In [54]:
# Opening JSON file
f = open('twitter_aapl_1.json',)
  
# returns JSON object as 
# a dictionary
aapl = json.load(f)

f.close()

In [55]:
flataapl  = [val for sublist in aapl for val in sublist]

In [56]:
print(flataapl[0])

{'created_at': '2021-05-11T10:18:35.000Z', 'id': '1392061565185236994', 'text': 'RT @Nicochan33: Apple Execs Chose to Keep a Hack of 128 Million iPhones Quiet https://t.co/waR9tgHKCA #tech #feedly #apple #iphone #cyberse…'}


In [57]:
df = pd.DataFrame(flataapl)
df

,created_at,id,text,withheld
0,2021-05-11T10:18:35.000Z,1392061565185236994,RT @Nicochan33: Apple Execs Chose to Keep a Ha...,NaN
1,2021-05-11T10:18:30.000Z,1392061546751217671,"RT @RoseLoveStyle: House of the Dragon, the pr...",NaN
2,2021-05-11T10:18:25.000Z,1392061526614360066,RT @gtorges: Ich habe jetzt einiges an Materia...,NaN
3,2021-05-11T10:17:54.000Z,1392061393415909376,RT @iTech911: Future versions of #Apple's CarK...,NaN
4,2021-05-11T10:17:48.000Z,1392061371886485506,発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #...,NaN
...,...,...,...,...
517126,2021-02-18T07:25:14.000Z,1362302135333969929,RT @mobitrade_: Original Apple Airpods Pro New...,NaN
517127,2021-02-18T07:25:08.000Z,1362302111027929089,#ios #Apple #swiftui \nSwiftUI 2.0 Complex UI ...,NaN
517128,2021-02-18T07:25:06.000Z,1362302102404542467,Sommige appels worden te snel slecht.\n\nhttps...,NaN
517129,2021-02-18T07:24:56.000Z,1362302061589708805,#apple #kia #StockMarkets https://t.co/ccRBBAKDtd,NaN


In [58]:
y = df['withheld'].dropna()
y

152369    {'copyright': False, 'country_codes': ['IN']}
257375    {'copyright': False, 'country_codes': ['IN']}
269997    {'copyright': False, 'country_codes': ['IN']}
315545    {'copyright': False, 'country_codes': ['RU']}
315578    {'copyright': False, 'country_codes': ['RU']}
343775    {'copyright': False, 'country_codes': ['IN']}
473481    {'copyright': False, 'country_codes': ['RU']}
473719    {'copyright': False, 'country_codes': ['RU']}
489545    {'copyright': False, 'country_codes': ['IN']}
509238    {'copyright': False, 'country_codes': ['IN']}
Name: withheld, dtype: object

In [59]:
df1 = df.drop('withheld',axis=1)
df1['created_at'] = df1['created_at'].map(lambda x: str(x)[:-5])
df1

,created_at,id,text
0,2021-05-11T10:18:35,1392061565185236994,RT @Nicochan33: Apple Execs Chose to Keep a Ha...
1,2021-05-11T10:18:30,1392061546751217671,"RT @RoseLoveStyle: House of the Dragon, the pr..."
2,2021-05-11T10:18:25,1392061526614360066,RT @gtorges: Ich habe jetzt einiges an Materia...
3,2021-05-11T10:17:54,1392061393415909376,RT @iTech911: Future versions of #Apple's CarK...
4,2021-05-11T10:17:48,1392061371886485506,発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #...
...,...,...,...
517126,2021-02-18T07:25:14,1362302135333969929,RT @mobitrade_: Original Apple Airpods Pro New...
517127,2021-02-18T07:25:08,1362302111027929089,#ios #Apple #swiftui \nSwiftUI 2.0 Complex UI ...
517128,2021-02-18T07:25:06,1362302102404542467,Sommige appels worden te snel slecht.\n\nhttps...
517129,2021-02-18T07:24:56,1362302061589708805,#apple #kia #StockMarkets https://t.co/ccRBBAKDtd


In [60]:
df1.to_csv("twitter_aapl_1_clean.csv", index = False)

In [2]:
df1 = pd.read_csv("twitter_aapl_1_clean.csv")
# Only including this step for now, basically removes the time component, leaving only date
df1['created_at'] = df1['created_at'].map(lambda x: str(x)[:-9])
df1 = df1.rename(columns={"created_at" : "Date"})
df1

,Date,id,text
0,2021-05-11,1392061565185236994,RT @Nicochan33: Apple Execs Chose to Keep a Ha...
1,2021-05-11,1392061546751217671,"RT @RoseLoveStyle: House of the Dragon, the pr..."
2,2021-05-11,1392061526614360066,RT @gtorges: Ich habe jetzt einiges an Materia...
3,2021-05-11,1392061393415909376,RT @iTech911: Future versions of #Apple's CarK...
4,2021-05-11,1392061371886485506,発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #...
...,...,...,...
517126,2021-02-18,1362302135333969929,RT @mobitrade_: Original Apple Airpods Pro New...
517127,2021-02-18,1362302111027929089,#ios #Apple #swiftui \nSwiftUI 2.0 Complex UI ...
517128,2021-02-18,1362302102404542467,Sommige appels worden te snel slecht.\n\nhttps...
517129,2021-02-18,1362302061589708805,#apple #kia #StockMarkets https://t.co/ccRBBAKDtd


In [3]:
# adapted from https://www.30secondsofcode.org/python/s/is-weekend
def if_weekend(d = datetime.today()):
    if d.weekday() == 5:
        da = d + timedelta(days=2)
        return da
    elif d.weekday() == 6:
        da = d + timedelta(days=1)
        return da
    elif d == date(2021,4,2):
        return date(2021,4,5)
    else:
        return d
print(if_weekend(date(2021,6,14)))
print(type(date(2021,6,14)))

2021-06-14
<class 'datetime.date'>


In [4]:
df1['Date']= pd.to_datetime(df1['Date'])
df1['usedate'] = df1['Date'].map(lambda x: if_weekend(x))
df1

,Date,id,text,usedate
0,2021-05-11,1392061565185236994,RT @Nicochan33: Apple Execs Chose to Keep a Ha...,2021-05-11
1,2021-05-11,1392061546751217671,"RT @RoseLoveStyle: House of the Dragon, the pr...",2021-05-11
2,2021-05-11,1392061526614360066,RT @gtorges: Ich habe jetzt einiges an Materia...,2021-05-11
3,2021-05-11,1392061393415909376,RT @iTech911: Future versions of #Apple's CarK...,2021-05-11
4,2021-05-11,1392061371886485506,発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #...,2021-05-11
...,...,...,...,...
517126,2021-02-18,1362302135333969929,RT @mobitrade_: Original Apple Airpods Pro New...,2021-02-18
517127,2021-02-18,1362302111027929089,#ios #Apple #swiftui \nSwiftUI 2.0 Complex UI ...,2021-02-18
517128,2021-02-18,1362302102404542467,Sommige appels worden te snel slecht.\n\nhttps...,2021-02-18
517129,2021-02-18,1362302061589708805,#apple #kia #StockMarkets https://t.co/ccRBBAKDtd,2021-02-18


In [5]:
price = pd.read_csv("aapl2.csv")
price['Date'] = price['Date'].map(lambda x: parser.parse(x).isoformat())
# Only including this step for now, basically removes the time component, leaving only date
price['Date'] = price['Date'].map(lambda x: str(x)[:-9])
price['Date']= pd.to_datetime(price['Date'])
# Change me if looking to change examination scope
price = price.rename(columns={"Date" : "usedate"})
price

,usedate,Close/Last,Volume,Open,High,Low,Change
0,2021-05-11,125.9100,126142800,123.5000,126.2700,122.7700,2.4100
1,2021-05-10,126.8500,88071230,129.4100,129.5400,126.8100,-2.5600
2,2021-05-07,130.2100,78973270,130.8500,131.2582,129.4750,-0.6400
3,2021-05-06,129.7400,78128330,127.8900,129.7500,127.1300,1.8500
4,2021-05-05,128.1000,84000900,129.2000,130.4500,127.9700,-1.1000
...,...,...,...,...,...,...,...
247,2020-05-18,78.7400,135372520,78.2925,79.1250,77.5810,0.4475
248,2020-05-15,76.9275,166348360,75.0875,76.9750,75.0525,1.8400
249,2020-05-14,77.3850,158929080,76.1275,77.4475,75.3825,1.2575
250,2020-05-13,76.9125,200622560,78.0375,78.9875,75.8025,-1.1250


In [6]:
merged = pd.merge(df1, price, how='left', on='usedate')
merged

,Date,id,text,usedate,Close/Last,Volume,Open,High,Low,Change
0,2021-05-11,1392061565185236994,RT @Nicochan33: Apple Execs Chose to Keep a Ha...,2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
1,2021-05-11,1392061546751217671,"RT @RoseLoveStyle: House of the Dragon, the pr...",2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
2,2021-05-11,1392061526614360066,RT @gtorges: Ich habe jetzt einiges an Materia...,2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
3,2021-05-11,1392061393415909376,RT @iTech911: Future versions of #Apple's CarK...,2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
4,2021-05-11,1392061371886485506,発売日に予約してようやく届いた🤗\n#Apple #airtag #ケースは純正じゃない #...,2021-05-11,125.91,126142800,123.5,126.270,122.77,2.41
...,...,...,...,...,...,...,...,...,...,...
517126,2021-02-18,1362302135333969929,RT @mobitrade_: Original Apple Airpods Pro New...,2021-02-18,129.71,96856750,129.2,129.995,127.41,0.51
517127,2021-02-18,1362302111027929089,#ios #Apple #swiftui \nSwiftUI 2.0 Complex UI ...,2021-02-18,129.71,96856750,129.2,129.995,127.41,0.51
517128,2021-02-18,1362302102404542467,Sommige appels worden te snel slecht.\n\nhttps...,2021-02-18,129.71,96856750,129.2,129.995,127.41,0.51
517129,2021-02-18,1362302061589708805,#apple #kia #StockMarkets https://t.co/ccRBBAKDtd,2021-02-18,129.71,96856750,129.2,129.995,127.41,0.51


In [7]:
merged.isna().sum()

Date          0
id            0
text          0
usedate       0
Close/Last    0
Volume        0
Open          0
High          0
Low           0
Change        0
dtype: int64

In [24]:
# merged.to_csv("merged.csv", index = False)

# Start Here

In [3]:
merged = pd.read_csv('mergedfull.csv')
from sklearn.model_selection import train_test_split  
mergedbig, mergedsmall = train_test_split(merged, test_size=0.15, random_state=0)
mergedsmall.to_csv("mergedsmall.csv", index = False)
mergedsmall = pd.read_csv('mergedsmall.csv')
merged = mergedsmall
# Obtaining the tweet contents into a list
all_tweets = merged["text"]
print(all_tweets)
all_tweets = all_tweets.to_list()
print(all_tweets[0])
print(all_tweets[1])
print(all_tweets[2])

0         2020 is almost ended\n\nBRAND NEW SEALED FACTO...
1         If you feel like Apple has no love for you by ...
2         📢NEW RIP IS LIVE📢\n\n2W1B 051 - iPhone, Capita...
3         #Apple has introduced a new #macOS version of ...
4         Good Deal😍 #MacBookPro #AppleEvent #Apple\n\nN...
                                ...                        
508177    Foreign Exchange History: How it All Started?\...
508178    Tropicana Apple Juice, 10 oz., 24 Count\n\nSta...
508179    "emotional conclusion to a dazzling series!"\n...
508180    3％超上昇した銘柄\n4434  サーバーワークス\n4625  アトミクス\n6777  ...
508181    EP 42: @seanmagers and @keithrconrad discuss t...
Name: text, Length: 508182, dtype: object
2020 is almost ended

BRAND NEW SEALED FACTORY UNLOCKED iPHONE12 Mini to one LUCKY FOLLOWER.

Keep Following &amp; Retweeting.

#iTech911 #iBuy #iSell #iSwap #iFix #APPLE 

0262666226 💬 📞 https://t.co/3vOZQIN8yz
If you feel like Apple has no love for you by not delivering the Heart Month Challeng

In [4]:
print(len(all_tweets))

508182


In [5]:
import nltk
# nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer = WordNetLemmatizer()
def lemming(tweet):
    a = word_tokenize(tweet)
    answer = list(map(lambda x: lemmatizer.lemmatize(x), a))
    return answer
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [6]:
# https://python.gotrained.com/scraping-tweets-sentiment-analysis/
# Basic cleaning of text before TF-IDF; this process will be improved later
# for tweet in all_tweets:
#     # Remove all the special characters
#     processed_tweet = re.sub(r'\W', ' ', tweet)
 
#     # remove all single characters
#     processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
 
#     # Remove single characters from the start
#     processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet) 
 
#     # Substituting multiple spaces with single space
#     processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)
 
#     # Removing prefixed 'b'
#     processed_tweet = re.sub(r'^b\s+', '', processed_tweet)
    
#     #removing common pronouns and prepositions
#     processed_tweet = re.sub(r'of|to|https|keep|128', '', processed_tweet)
#     processed_tweet = lemmatizer.lemmatize(processed_tweet)
# #     lambda x: lemmatizer.lemmatize(x in processed_tweet)
#     # Converting to Lowercase
#     processed_tweet = processed_tweet.lower()
    
#     tweet = processed_tweet
    
    
processed_tweets = []
X = all_tweets
for tweet in range(0, len(X)):  
    

    processed_tweet = re.sub(r"http\S+", "", str(X[tweet]))

    # Remove all the special characters
    processed_tweet = re.sub(r'\W', ' ', processed_tweet)
    
#     processed_tweet = re.sub(r'http\S+', '', processed_tweet)
    
    
#     processed_tweet = re.sub(r'co\S+', '', processed_tweet) 
    # remove all single characters
    processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
 
    # Remove single characters from the start
    processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet) 
 
    # Substituting multiple spaces with single space
    processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)
 
    # Removing prefixed 'b'
    processed_tweet = re.sub(r'^b\s+', '', processed_tweet)
    

    
#     processed_tweet = re.sub(r'of|to|https|keep', '', processed_tweet)
    # Converting to Lowercase
    processed_tweet = processed_tweet.lower()
    
    processed_tweet = lemming(processed_tweet)
    processed_tweet = TreebankWordDetokenizer().detokenize(processed_tweet)    
    
    # remove all single characters
    processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
    processed_tweets.append(processed_tweet)

In [22]:
print(processed_tweets[0])
print(processed_tweets[1])
print(processed_tweets[2])

2020 is almost ended brand new sealed factory unlocked iphone12 mini to one lucky follower keep following amp retweeting itech911 ibuy isell iswap ifix apple 0262666226
if you feel like apple ha no love for you by not delivering the heart month challenge badge just restart your watch and wait for few minute you ll get it apple applewatch challenge
new rip is live 2w1b 051 iphone capitalism amazon tax gripe mondaythoughts mondaymorning mondaymotivation hamont btc bitcoin iphone12 apple aapl eeftober spotify ps5 investing itunes sony weed cgc etf amzn spot nlpoli


In [6]:

def identity_tokenizer(text):
    return text
tfidfv = TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')
df2 = tfidfv.fit_transform(processed_tweets)
print(df2)
# df2array = df2.toarray()
# print(df2array)

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


  (0, 12)	0.10378008268576916
  (0, 34)	0.18419582908637713
  (0, 14)	0.08445848860963202
  (0, 30)	0.09187542928543256
  (0, 26)	0.22443153601406954
  (0, 22)	0.08463426568468516
  (0, 9)	0.2547065677027308
  (0, 3)	0.17119712179269916
  (0, 2)	0.14122564458503742
  (0, 16)	0.17872517137676927
  (0, 20)	0.19891043303145406
  (0, 28)	0.1684994505688957
  (0, 33)	0.1818628830219665
  (0, 15)	0.27180525325058125
  (0, 21)	0.1933259623804885
  (0, 25)	0.22012294324860932
  (0, 4)	0.36187297077110625
  (0, 18)	0.24219945381211117
  (0, 24)	0.22511263095002002
  (0, 13)	0.25298058821310954
  (0, 23)	0.20366114070115587
  (0, 0)	0.27999544406345467
  (0, 29)	0.18492188932933612
  (0, 27)	0.13048352030712054
  (1, 32)	0.1116202834795065
  :	:
  (517130, 11)	0.135839254980182
  (517130, 10)	0.38618704858746866
  (517130, 17)	0.10080806835159276
  (517130, 12)	0.13581868475479603
  (517130, 34)	0.11486642069017963
  (517130, 14)	0.12568418027292302
  (517130, 30)	0.12023887092682785
  (517130, 

In [21]:
print(type(df2))
print(type(df2[-1]))
print(type(df2[-1][-1]))
print(df2[-1][-1])

<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
  (0, 556)	0.40357727068148264
  (0, 88)	0.34308710016633726
  (0, 97)	0.417333363196858
  (0, 443)	0.3951839877481259
  (0, 674)	0.3663719517401211
  (0, 1770)	0.3199229308458355
  (0, 1783)	0.31050472625527564
  (0, 943)	0.15704072326243243
  (0, 1716)	0.15129041849415864
  (0, 1471)	0.09239185493528781


In [11]:
print(df2.size)

6813202


In [12]:
print(df2[0])

print(df2array[0])
print(type(df2[0][0]))

  (0, 943)	0.1619164850216469
  (0, 1667)	0.24642359750991072
  (0, 412)	0.07984327940748012
  (0, 863)	0.07926270652557654
  (0, 957)	0.3644418579168204
  (0, 1148)	0.37252279088881424
  (0, 24)	0.44364094790152897
  (0, 1243)	0.19111331976332038
  (0, 802)	0.4405447542842643
  (0, 1005)	0.38259748006594846
  (0, 1716)	0.15598764620493955
  (0, 189)	0.14697293203863546
  (0, 1471)	0.09526041452797107
[0. 0. 0. ... 0. 0. 0.]
<class 'scipy.sparse.csr.csr_matrix'>


In [ ]:
print(type(df2array))
np.set_printoptions(threshold=sys.maxsize)
print(df2array[0])
np.set_printoptions(threshold = False)

In [20]:
print(processed_tweets[0:10])

['rt nicochan33 apple exec chose to keep hack of 128 million iphones quiet tech feedly apple iphone cyberse', 'rt roselovestyle house of the dragon the prequel to game of throne wa announced by casey bloys president of hbo gamesofthrones', 'rt gtorges ich habe jetzt einiges an material zu fuellmich amp co beim applesupport eingereicht hoffentlich verschwindet der podcast', 'rt itech911 future version of apple carkey could detect when it being used near wireless charger and alter how it work to avoid', '発売日に予約してようやく届いた apple airtag ケースは純正じゃない 忘れん坊 実は追加でもう一つ買った', 'rt nicochan33 apple exec chose to keep hack of 128 million iphones quiet tech feedly apple iphone cyberse', 'rt foxconcours_ apple macbook tente de gagner un macbookpro13 en participant ce concours pour tenter votre chance rt fo', 'rt taisy0 apple apple tv でsiriを利用可能な国を拡大 気になる 記になる apple', 'rt neodiycom how to fix apple macbookpro 2011 running windows10 10 64 bit audio sound driver problem macbook', 'rt indievideogames dino mak

In [22]:
print(len(processed_tweets))
print(type(processed_tweets[0]))

517131
<class 'str'>


In [15]:
print(type(df2))

<class 'scipy.sparse.csr.csr_matrix'>


In [6]:
# https://programmerbackpack.com/tf-idf-explained-and-python-implementation/
dfnew = pd.DataFrame(df2[0].T.todense(), index=tfidfv.get_feature_names(), columns=["TF-IDF"])
dfnew = dfnew.sort_values('TF-IDF', ascending=False)
print (dfnew.head(30))

NameError: name 'df2' is not defined

In [39]:
# Trying an alternative data implementation, I haven't used this in the later stages yet
df3 = pd.DataFrame.sparse.from_spmatrix(df2)
print(type(df3))
df3

<class 'pandas.core.frame.DataFrame'>


,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
517127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
517128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
517129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
#Selecting the data and splitting into train and test
y = merged['Change']
print(type(y))
# X = df3
# X = df2
# X = df2array
X = processed_tweets
from sklearn.model_selection import train_test_split  
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)
# train_data = train_data.sample(frac=x)
# train_holdout_data.to_csv('train_holdout_data.csv', index=False)
# x_holdout.to_csv('x_holdout.csv', index=False)
# y_holdout.to_csv('y_holdout.csv', index=False)
# train_data.to_csv('train_data.csv', index=False)
# # x_train.to_csv('x_train.csv', index=False)
# y_train.to_csv('y_train.csv', index=False)
# # x_test.to_csv('x_test.csv', index=False)
# y_test.to_csv('y_test.csv', index=False)

# # train_holdout_data = pd.read_csv('train_holdout_data.csv')
# # x_holdout = pd.read_csv('x_holdout.csv')
# # y_holdout = pd.read_csv('y_holdout.csv')
# # x_train = pd.read_csv('x_train.csv')
# y_train = pd.read_csv('y_train.csv')
# # x_test = pd.read_csv('x_test.csv')
# y_test = pd.read_csv('y_test.csv')


<class 'pandas.core.series.Series'>


In [11]:
# y_train = pd.Series(y_train)
# y_test = pd.Series(y_test)
print(len(x_train))
# print(type(y_train.values))
# print(np.isnan(X).sum())
# print(y.isna().sum())

406545


In [14]:
# x_train = torch.from_numpy(x_train)
# x_test = torch.from_numpy(x_test)
# y_train = torch.from_numpy(y_train.values)
# y_test = torch.from_numpy(y_test.values)
print(len(x_train))
# print(x_train)
# print(x_test)
# print(y_train)
# print(y_test)

413704


In [12]:
print(type(x_train))
print(type(x_test))
print(type(y_train))
print(type(y_test))
print(x_train.dtype)
print(x_test.dtype)
print(y_train.dtype)
print(y_test.dtype)
# print(x_test[100000][0])
# if 'X' in x_train:
#     print("hi")

<class 'list'>
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


AttributeError: 'list' object has no attribute 'dtype'

In [20]:
# y_train = y_train.astype(np.float32)
# y_test = y_test.astype(np.float32)
y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)
# y_train = y_train.squeeze()
# y_test = y_test.squeeze()

In [13]:
print(y_train.ndim)

2


In [43]:
# x_train1 = x_train.astype(np.float32)
# y_train1 = y_train.astype(np.float32)
print(x_train.dtype)
print(x_test.dtype)
print(y_train.dtype)
print(y_test.dtype)
print(x_train1.dtype)
print(y_train1.dtype)

AttributeError: 'list' object has no attribute 'dtypes'

In [45]:
# for x in y_train[0]:
#     if type(x) != "class 'numpy.float32'":
#         print("double")
print(type(x_train[0]))
print(type(y_train))
print(type(y_train[0]))
print(type(y_train[0][0]))

<class 'str'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.float32'>


AttributeError: 'numpy.ndarray' object has no attribute 'float'

In [20]:
y_trainarray = y_train.values.tolist()
y_testarray = y_test.values.tolist()
# y_trainarray = y_train.to_numpy()
# y_testarray = y_test.to_numpy()

In [13]:
def identity_tokenizer(text):
    return text

tfidfv = TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')
x_train1 = tfidfv.fit_transform(x_train)
x_test1 = tfidfv.fit_transform(x_test)
x_train2 = x_train1.astype(dtype = np.float32)
x_test2 = x_test1.astype(dtype = np.float32)

In [14]:
print(x_train2.shape[1])
print(y_train.shape)
print(y_train)

5205
(413704, 1)
[[ 1.33]
 [-2.01]
 [ 1.14]
 ...
 [-3.29]
 [-2.73]
 [-1.33]]


In [33]:
print(len(x_train))
# tokenise me pls
a = [word_tokenize(x) for x in x_train]

In [39]:
def flatten(t):
    return [item for sublist in t for item in sublist]
print(len(set(flatten(a))))

373108


In [9]:
# import torch.nn.functional as F
from nltk.tokenize.treebank import TreebankWordDetokenizer
def identity_tokenizer(text):
    return text

in_dimension = 413704
hid_dimension = 3
out_dimension = 1


class PoleNN(nn.Module):
    def __init__(self):
        super(PoleNN, self).__init__()
        self.fc1 = nn.Linear(in_dimension,hid_dimension)
        self.fc2 = nn.Linear(hid_dimension,out_dimension)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, X):
        hidden = self.fc1(X)
        hidden = self.sigmoid(hidden)
        output = self.fc2(hidden)
        return output

from skorch import NeuralNetRegressor
from sklearn.pipeline import Pipeline
x_trainshape = 5842
class RegressorModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
    ):
        super(RegressorModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(x_trainshape, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = F.relu(self.dense1(X))
        X = self.output(X)
        return X

pole_model = RegressorModule()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(pole_model.parameters(), lr = 0.1)

net = NeuralNetRegressor(module=pole_model, max_epochs=30, lr=0.1, callbacks =[('earlystopping',EarlyStopping())])
def typechange(x):
    return x.astype(dtype = np.float32)
typetransform = FunctionTransformer(typechange)
def inputneuron(x):
    x_trainshape = x.shape[1]
#     return x_trainshape
inputneuronnumber = FunctionTransformer(inputneuron)
# pipe = Pipeline([('transform', DataFrameTransformer()),('net', net)])
# pipe = Pipeline([('net', net)])
pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')), ("typetransform", typetransform), ("net", net)])


# net = skorch.NeuralNetClassifier(module=PoleNN, max_epochs=20, lr=0.1, criterion=torch.nn.NLLLoss)
pipe.fit(X=x_train, y=y_train)
y_pred = pipe.predict(x_test)

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        4.1718        4.0988  74.6012
      2        4.0228        4.0428  70.6258
      3        3.9905        4.0125  68.7844
      4        3.9709        3.9934  69.1349
      5        3.9554        3.9805  69.8406
      6        3.9436        3.9816  68.5065
      7        3.9315        3.9778  68.6152
      8        3.9206        3.9605  68.8205
      9        3.9104        3.9722  69.0376
     10        3.9013        3.9433  68.9242
     11        3.8915        3.9386  68.4290
     12        3.8838        3.9411  69.4425
     13        3.8770        3.9544  75.4372
     14        3.8697        3.9361  70.9167
     15        3.8632        3.9429  67.9107
     16        3.8576        3.9392  68.3335
     17        3.8512        3.9534  69.3909
     18        3.8461        3.9376  71.2812
Stopping since valid_loss has not improved in the last 5 epochs.


In [10]:
rmse = mean_squared_error(y_test, y_pred, squared = False)
rmse

1.9780481

In [21]:
profit = 0
for i, (real, pred) in enumerate(zip(y_test, y_pred)):
    if i == 0:
        pass
    else:
        realchange = y_test[i] - y_test[i-1]
        invest  = (y_pred[i] - y_pred[i-1])
        change = (invest * realchange)
        profit += change
print(profit)
profit = 0           
for i, (real, pred) in enumerate(zip(y_test, y_pred)):
    realmean = y_test.mean()
    predmean = y_pred.mean()
    realchange = real - realmean
    predchange = pred - predmean
    change = (predchange * realchange)
    profit += change
print(profit)
            
       

[71518.68]
[35764.336]


In [13]:
from scipy.stats import loguniform, uniform
print(uniform(20, 50))

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_validate
from scipy.stats import loguniform, uniform
params = {
    'net__lr': [0.001,0.003,0.01,0.03,0.1,0.3,0.6,0.9,1],
    'net__max_epochs': [20,40,50],
    'net__optimizer__momentum': [0.95,0.96,0.97,0.98,0.99]
}

# specify parameters and distributions to sample from
param_dist = {'net__lr': loguniform(0.001,1),
              'net__max_epochs': [20,30,40,50],
              'net__optimizer__momentum': loguniform(0.9, 0.99)}

pipe = Pipeline([("typetransform", typetransform), ("net", net)])
tfidf = TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')
x_train2 = tfidf.fit_transform(x_train)
x_trainshape = x_train2.shape[1]
class RegressorModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
    ):
        super(RegressorModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(x_trainshape, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = F.relu(self.dense1(X))
        X = self.output(X)
        return X

pole_model = RegressorModule()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(pole_model.parameters(), lr = 0.1)

net = NeuralNetRegressor(module=pole_model, max_epochs=30, lr=0.1, callbacks =[('earlystopping',EarlyStopping())])
# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(pipe, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time.time()
random_search.fit(x_train2, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time.time() - start), n_iter_search))
report(random_search.cv_results_)


# gs = GridSearchCV(pipe, params, n_jobs=-1)
# gs.fit(X=x_train, y=y_train);
# print(gs.best_score_, gs.best_params_)

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_para

In [ ]:
dftest = pd.DataFrame(y_test)
dftest['index'] = list(range(103427))
dfpred = pd.DataFrame(y_pred)
dfpred['index'] = list(range(103427))
fix, ax = plt.subplots()
sns.kdeplot(x=dftest['index'],y=dftest[0], cumulative=True, color='orange', label='real')
sns.kdeplot(x=dfpred['index'],y=dfpred[0], cumulative=True, color='b', label='pred')

plt.show()

In [48]:
from sklearn.datasets import make_regression

X_regr, y_regr = make_regression(1000, 20, n_informative=10, random_state=0)
X_regr = X_regr.astype(np.float32)
y_regr = y_regr.astype(np.float32) / 100
y_regr = y_regr.reshape(-1, 1)

X_regr.shape, y_regr.shape, y_regr.min(), y_regr.max()

((1000, 20), (1000, 1), -6.4901485, 6.154505)

In [52]:
print(type(y_regr))
print(type(y_regr[0]))
print(type(y_regr[0][0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.float32'>


In [75]:
net.fit(X=X_regr, y=y_regr)

Re-initializing module.
Re-initializing optimizer.
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        4.6416        3.8738  1.3260
      2        4.3003        2.7721  0.0300
      3        1.7503        0.4882  0.0240
      4        0.4048        0.1588  0.0240
      5        0.1568        0.2447  0.0240
      6        0.2712        0.1374  0.0260
      7        0.1773        0.4521  0.0220
      8        0.2750        0.0977  0.0250
      9        0.0932        0.1167  0.0230
     10        0.0787        0.0926  0.0280


<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=RegressorModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=1, bias=True)
  ),
)

In [ ]:

from sklearn.ensemble import RandomForestRegressor
text_classifier = RandomForestRegressor(n_estimators=100, random_state=0)  
text_classifier.fit(x_train, y_train)


predictions = text_classifier.predict(x_test)
 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
 
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

In [73]:
cross_val_score.get_params()

AttributeError: 'function' object has no attribute 'get_params'

In [8]:
y = merged['Change']

X = processed_tweets

data = pd.DataFrame()
data['text'] = processed_tweets
data['y'] = y.tolist()


from sklearn.model_selection import train_test_split  
data1 = data.sample(frac=0.2)
X1 = data1['text'].tolist()
y2 = np.array(data1['y'])
x_trainsvm, x_testsvm, y_trainsvm, y_testsvm = train_test_split(X1, y2, test_size=0.2, random_state=0)


In [9]:
from sklearn.svm import LinearSVR, SVR

def identity_tokenizer(text):
    return text


# y_trainsvm = np.array(y_train).ravel()
# y_testsvm = np.array(y_test).ravel()

#Create the SVM model

start = time.time()
# regressor = LinearSVR(random_state = 0)
regressor = SVR()
#Fit the model for the data
pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')), ("regressor", regressor)])

pipe.fit(x_trainsvm, y_trainsvm)

#Make the prediction
y_predsvm = pipe.predict(x_testsvm)

end = time.time()
print("Elapsed Time: ", (end-start),"\n")

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Elapsed Time:  1705.32363986969 



In [11]:
rmse = mean_squared_error(y_testsvm, y_predsvm, squared = False)
print("RMSE:",rmse)
r2 = r2_score(y_testsvm,y_predsvm)
print("R-Squared:",r2)

RMSE: 1.977553460848178
R-Squared: 0.0333595799537606


In [ ]:
accuracies = cross_val_score(estimator = regressor, X = x_trainsvm, y = y_trainsvm, cv = 10)
end = time.time()
# print("Fraction: ", x)
# print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
# print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))
mse = mean_squared_error(y_test,y_predsvm)
print("Mean Squared Error: {}".format(mse))
r2 = r2_score(y_test,y_predsvm)
print("R-squared: {}".format(r2))
print("")
print("Elapsed Time: ", (end-start),"\n")

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [31]:
mse = mean_squared_error(y_test,y_predsvm)
print("Mean Squared Error: {}".format(mse))
r2 = r2_score(y_test,y_predsvm)
print("R-squared: {}".format(r2))

4.00098072231255
Mean Squared Error: 4.00098072231255
R-squared: 0.025518011281076225
